# 三国演义 分析

## 准备工作

### 安装依赖
- Python=3.11
- HanLP
- NumPy
- Pytorch

### 读取源数据

In [14]:
import os

dataset_src = []
if os.path.exists("dataset/三国演义.txt"):
    with open("dataset/三国演义.txt", "r", encoding="utf-8-sig") as f:
        dataset_src = f.readlines()  # 逐行读取
else:
    print("文件不存在")

### 数据预处理

In [44]:
import json
import re

dataset = []
# 去空行
dataset = [line.strip() for line in dataset_src if line != "\n"]
# 去除额外信息
dataset = dataset[3:-2]
chapter_index = []
chapter_content = [[]]
chapter_cnt = 0
chinese_punctuation = r"[,.!;:，。！：；—　。，、》《？”“]"  # 中文标点符号
for i in range(len(dataset)):
    if dataset[i].startswith("正文"):
        chapter_cnt += 1
        chapter_index.append(i)  # 获取章节索引
        chapter_content.append([])  # 新建章节
    else:
        # dataset[i] = re.sub(chinese_punctuation, '', dataset[i])  # 去除标点符号
        chapter_content[chapter_cnt].append(dataset[i])  # 获取内容
        # 去除标点符号

# 保存为json
data = []
for i in range(len(chapter_index)):
    line = dataset[chapter_index[i]]
    data.append(
        {
            "chapter": line[line.find("回") + 1 :].strip(),
            "content": chapter_content[i + 1],
        }
    )
with open("dataset/dataset.json", "w", encoding="utf-8-sig") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

# Task 0

### 导入预处理数据集

In [2]:
import json

data = []
with open("dataset/dataset.json", "r", encoding="utf-8-sig") as f:
    data = json.load(f)
print("数据集大小：", len(data))

数据集大小： 120


## Task 1

### 获取章回

In [26]:
import tqdm

chapters = []
for i in tqdm.tqdm(range(len(data))):
    chapters.append(data[i]["chapter"])

100%|██████████| 120/120 [00:00<00:00, 787662.72it/s]


### 保存

In [38]:
import csv

with open("export/chapters.csv", "w", newline="", encoding="utf-8-sig") as csvfile:
    writer = csv.writer(csvfile)
    for i in range(len(chapters)):
        writer.writerow([chapters[i]])

### 输出

In [27]:
print("共{}章".format(len(chapters)))
for i in range(len(chapters)):
    print("第{}章：{}".format(i + 1, chapters[i]))

共120章
第1章：宴桃园豪杰三结义 斩黄巾英雄首立功
第2章：张翼德怒鞭督邮 何国舅谋诛宦竖
第3章：议温明董卓叱丁原 馈金珠李肃说吕布
第4章：废汉帝陈留践位 谋董贼孟德献刀
第5章：发矫诏诸镇应曹公 破关兵三英战吕布
第6章：焚金阙董卓行凶 匿玉玺孙坚背约
第7章：袁绍磐河战公孙 孙坚跨江击刘表
第8章：王司徒巧使连环计 董太师大闹凤仪亭
第9章：除暴凶吕布助司徒 犯长安李傕听贾诩
第10章：勤王室马腾举义 报父仇曹操兴师
第11章：刘皇叔北海救孔融 吕温侯濮阳破曹操
第12章：陶恭祖三让徐州 曹孟穗大战吕布
第13章：李傕郭汜大交兵 杨奉董承双救驾
第14章：曹孟德移驾幸许都 吕奉先乘夜袭徐郡
第15章：太史慈酣斗小霸王 孙伯符大战严白虎
第16章：吕奉先射戟辕门 曹孟德败师淯水
第17章：袁公路大起七军 曹孟德会合三将
第18章：贾文和料敌决胜 夏侯惇拨矢啖睛
第19章：下邳城曹操鏖兵 白门楼吕布殒命
第20章：曹阿瞒许田打围 董国舅内阁受诏
第21章：曹操煮酒论英雄 关公赚城斩车胄
第22章：袁曹各起马步三军 关张共擒王刘二将
第23章：祢正平裸衣骂贼 吉太医下毒遭刑
第24章：国贼行凶杀贵妃 皇叔败走投袁绍
第25章：屯土山关公约三事 救白马曹操解重围
第26章：袁本初败兵折将 关云长挂印封金
第27章：美髯公千里走单骑 汉寿侯五关斩六将
第28章：斩蔡阳兄弟释疑 会古城主臣聚义
第29章：小霸王怒斩于吉 碧眼儿坐领江东
第30章：战官渡本初败绩 劫乌巢孟德烧粮
第31章：曹操仓亭破本初 玄德荆州依刘表
第32章：夺冀州袁尚争锋 决漳河许攸献计
第33章：曹丕乘乱纳甄氏 郭嘉遗计定辽东
第34章：蔡夫人隔屏听密语 刘皇叔跃马过檀溪
第35章：玄德南漳逢隐沧 单福新野遇英主
第36章：玄德用计袭樊城 元直走马荐诸葛
第37章：司马徽再荐名士 刘玄德三顾草庐
第38章：定三分隆中决策 战长江孙氏报仇
第39章：荆州城公子三求计 博望坡军师初用兵
第40章：蔡夫人议献荆州 诸葛亮火烧新野
第41章：刘玄德携民渡江 赵子龙单骑救主
第42章：张翼德大闹长坂桥 刘豫州败走汉津口
第43章：诸葛亮舌战群儒 鲁子敬力排众议
第44章：孔明用智激周瑜 孙权决计破曹操
第45章：三江口曹操折兵 群英会蒋干中计
第46章：用奇谋孔明借箭 献密计黄盖受刑
第47章：阚泽密献

# Task 2

### 进行分析

- 加载hanlp模型

In [ ]:
import hanlp
import tqdm

HanLP = hanlp.load(
    hanlp.pretrained.mtl.CLOSE_TOK_POS_NER_SRL_DEP_SDP_CON_ELECTRA_BASE_ZH
)

/mnt/workspace_ssd/r3k_analysis/.conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


- 使用hanlp进行分词并统计

In [14]:
person = dict()
for i in tqdm.tqdm(range(len(data))):
    chapter = [data[i]["chapter"]]
    for j in data[i]["content"]:
        chapter.append(j)
    for j in range(len(chapter)):
        result = HanLP(chapter[j], tasks="ner/msra")
        for k in range(len(result["ner/msra"])):
            if result["ner/msra"][k][1] == "PERSON":
                word = result["ner/msra"][k][0]
                if word in person:
                    person[word]["count"] += 1  # 计数
                else:
                    # 初始化
                    person[word] = {}
                    person[word]["position"] = []
                    person[word]["count"] = 1
                person[word]["position"].append((i + 1, j, k + 1))  # 以三维点集记录位置

100%|██████████| 120/120 [01:38<00:00,  1.22it/s]


### 读取人名映射

In [93]:
import json

person_map = {}
with open("dataset/map.json", "r", encoding="utf-8-sig") as f:
    map_json = json.load(f)
    for i in map_json.keys():
        person_map[i] = set()
        for j in map_json[i]:
            person_map[i].add(j)
person_dict = dict()
person_export = dict()
for i in person_map.keys():
    if i not in person_export:
        person_export[i] = {}
    person_export[i]["count"] = 0
    person_export[i]["position"] = set()
    for j in person_map[i]:
        person_dict[j] = i

### 合并

In [94]:
for i in person.keys():
    if i in person_dict:
        person_export[person_dict[i]]["count"] += person[i]["count"]
        for j in person[i]["position"]:
            person_export[person_dict[i]]["position"].add(j)
    else:
        if len(i) >= 2:
            person_export[i] = person[i]

### 排序处理

In [95]:
persons = []
for i in person_export.keys():
    pos = list(person_export[i]["position"])
    pos = sorted(pos, key=lambda x: x[0])
    persons.append([i, person_export[i]["count"], pos[0], pos])

persons = sorted(persons, key=lambda x: x[1], reverse=True)

### 存储结果

In [96]:
import csv

with open("export/persons.csv", "w", newline="", encoding="utf-8-sig") as csvfile:
    writer = csv.writer(csvfile)
    for i in persons:
        writer.writerow(i)

### 输出


In [97]:
print("{}是第一主角".format(persons[0][0]))
order = {}
for i in range(100):
    print("第{}名：{}，共{}次".format(i + 1, persons[i][0], persons[i][1]))
    order.update({persons[i][0]: persons[i][2]})
print("=" * 10)
print("出场顺序:")
sorted_order = sorted(order.items(), key=lambda x: (x[1][0], x[1][1]))
for name, position in sorted_order:
    print("{}:第{}章第{}句".format(name, position[0], position[1]))

曹操是第一主角
第1名：曹操，共2723次
第2名：刘备，共2105次
第3名：诸葛亮，共1932次
第4名：关羽，共846次
第5名：吕布，共673次
第6名：赵云，共585次
第7名：司马懿，共569次
第8名：周瑜，共566次
第9名：孙权，共536次
第10名：张飞，共534次
第11名：姜维，共532次
第12名：袁绍，共442次
第13名：魏延，共408次
第14名：马超，共322次
第15名：邓艾，共293次
第16名：董卓，共253次
第17名：张郃，共237次
第18名：黄忠，共234次
第19名：孙策，共227次
第20名：司马昭，共202次
第21名：张辽，共184次
第22名：徐晃，共183次
第23名：孟获，共182次
第24名：曹丕，共172次
第25名：庞统，共169次
第26名：曹仁，共162次
第27名：关兴，共157次
第28名：夏侯惇，共150次
第29名：陆逊，共148次
第30名：许褚，共145次
第31名：刘璋，共144次
第32名：刘表，共142次
第33名：关平，共141次
第34名：夏侯渊，共140次
第35名：曹睿，共139次
第36名：袁术，共134次
第37名：曹真，共130次
第38名：王允，共129次
第39名：鲁肃，共128次
第40名：曹洪，共126次
第41名：庞德，共126次
第42名：马岱，共126次
第43名：吕蒙，共120次
第44名：孙乾，共117次
第45名：张苞，共117次
第46名：太史慈，共106次
第47名：郭汜，共104次
第48名：王平，共101次
第49名：郭淮，共100次
第50名：廖化，共100次
第51名：诸葛瑾，共98次
第52名：张鲁，共97次
第53名：司马，共94次
第54名：孟达，共93次
第55名：周泰，共91次
第56名：李傕，共86次
第57名：甘宁，共85次
第58名：蔡瑁，共80次
第59名：马腾，共79次
第60名：李典，共78次
第61名：张翼，共78次
第62名：孙坚，共77次
第63名：荀彧，共73次
第64名：黄盖，共72次
第65名：程普，共71次
第66名：徐盛，共70次
第67名：严颜，共69次
第68名：明公，共68次
第69名：张昭，共64次
第70名：张任，共64次
第71名：颜良，共61次
第72名：诸葛，共60次
第73名：

# Task 3

### 读取 Task 2 中获取的数据并转换

In [130]:
import csv
import numpy as np

max_cnt=10
def parse(s):
    # 移除方括号并分割元组字符串
    s = s.strip("[]")
    tuple_strings = s.split("), (")

    result = []
    for t in tuple_strings:
        # 清理元组字符串
        t = t.strip("()")
        # 分割并转换为整数
        numbers = [int(x) for x in t.split(", ")]
        # 转换为元组并添加到结果列表
        result.append(tuple(numbers))
    return result


persons = {}
with open("export/persons.csv", "r", encoding="utf-8-sig") as csvfile:
    reader = csv.reader(csvfile)
    count = 0
    for row in reader:
        if count == max_cnt:
            break
        count += 1
        persons[row[0]] = []
        input_data = parse(row[3])
        for k in input_data:
            persons[row[0]].append(np.array(k))  # 读取位置信息并转换为向量

### 对数据进行加权

In [136]:
# 统计平均数量
avg_lines=0
avg_words=0
for i in range(len(data)):
    avg_lines+=len(data[i]["content"])
    for j in range(len(data[i]["content"])):
        avg_words+=len(data[i]["content"][j])
    avg_words/=len(data[i]["content"])
avg_lines/=len(data)
avg_words/=3
print("平均每章{}句，每句{}词".format(avg_lines,avg_words))
# 加权
for i in persons.keys():
    for j in range(len(persons[i])):
        persons[i][j][0]=persons[i][j][0]*avg_lines*avg_words
        persons[i][j][1]=persons[i][j][1]*avg_words

平均每章17.441666666666666句，每句60.53441924928282词


### 计算相关度
- 使用点集中的点到点集的最邻近距离计算

In [140]:
import torch
from tqdm import tqdm
import numpy as np


def calc(persons):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    dis = {}

    # 预处理：将numpy数组转换为单个tensor
    person_tensors = {
        k: torch.from_numpy(np.array(v, dtype=np.float32)).to(device)
        for k, v in persons.items()
    }

    for i in tqdm(person_tensors.keys()):
        for j in person_tensors.keys():
            if i == j:
                continue

            points_i = person_tensors[i]
            points_j = person_tensors[j]

            with torch.no_grad():
                # 计算距离
                distances = torch.cdist(points_i, points_j, p=1)
                min_distances = torch.min(distances, dim=1)[0]
                dis[(i, j)] = float(torch.mean(min_distances).cpu())

    return dis


if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True
    print("使用GPU加速计算")
else:
    print("使用CPU计算")

dis = calc(persons)

使用GPU加速计算


100%|██████████| 10/10 [00:00<00:00, 26.95it/s]


### 排序并储存

In [141]:
import csv

with open("export/distance.csv", "w", newline="", encoding="utf-8-sig") as csvfile:
    writer = csv.writer(csvfile)
    dis_sorted = sorted(dis.items(), key=lambda x: x[1])
    for relation, distance in dis_sorted:
        writer.writerow([relation[0], relation[1], distance])

### 输出

In [142]:
for i in range(30):
    print("第{}近的关系：{}和{}，距离为{}".format(i + 1, dis_sorted[i][0][0], dis_sorted[i][0][1], dis_sorted[i][1]))

第1近的关系：周瑜和曹操，距离为8720.037109375
第2近的关系：司马懿和诸葛亮，距离为23330.90625
第3近的关系：张飞和刘备，距离为26015.28125
第4近的关系：刘备和曹操，距离为61199.3828125
第5近的关系：张飞和曹操，距离为76769.015625
第6近的关系：关羽和曹操，距离为124450.4765625
第7近的关系：关羽和刘备，距离为206526.890625
第8近的关系：周瑜和刘备，距离为272244.625
第9近的关系：曹操和刘备，距离为276834.0
第10近的关系：刘备和张飞，距离为363248.78125
第11近的关系：吕布和曹操，距离为473303.75
第12近的关系：周瑜和孙权，距离为475874.78125
第13近的关系：刘备和关羽，距离为540640.9375
第14近的关系：孙权和曹操，距离为546298.8125
第15近的关系：赵云和曹操，距离为547632.0
第16近的关系：关羽和赵云，距离为730768.9375
第17近的关系：吕布和刘备，距离为742915.125
第18近的关系：张飞和关羽，距离为795325.75
第19近的关系：关羽和张飞，距离为822762.5
第20近的关系：吕布和关羽，距离为823639.5
第21近的关系：周瑜和张飞，距离为844390.75
第22近的关系：周瑜和赵云，距离为859221.6875
第23近的关系：孙权和刘备，距离为866656.9375
第24近的关系：周瑜和关羽，距离为904098.25
第25近的关系：张飞和赵云，距离为1011295.75
第26近的关系：曹操和张飞，距离为1048184.6875
第27近的关系：吕布和张飞，距离为1057627.25
第28近的关系：刘备和赵云，距离为1058997.5
第29近的关系：诸葛亮和孙权，距离为1106353.625
第30近的关系：曹操和关羽，距离为1146432.625


# Task 4

### 读取人物名单

In [143]:
import csv

person_list = []
with open("export/person.csv", "r", encoding="utf-8-sig") as csvfile:
    reader = csv.reader(csvfile)
    count = 0
    for row in reader:
        if count == 20:
            break
        count += 1
        person_list.append(row)


### 进行词向量转换

- 加载模型

In [1]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import tqdm
# 加载模型
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')


# 定义关键词
keywords = ["胜", "败", "克", "擒", "大败", "捷", "破", "取", "降"]
keyword_embeddings = model.encode(keywords)


/mnt/workspace_ssd/r3k_analysis/.conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


  0%|          | 0/120 [00:00<?, ?it/s]


ValueError: Expected 2D array, got 1D array instead:
array=[-2.45404154e-01  5.91705382e-01  5.33690393e-01  1.84479430e-02
 -5.04138470e-01 -2.06162915e-01  1.11922526e+00  1.89100757e-01
  6.99254453e-01 -4.34618235e-01  5.12407839e-01 -1.26707411e+00
  2.97268897e-01 -1.60574839e-01  2.99710155e-01 -1.45805731e-01
  2.10144714e-01  2.89475948e-01 -7.56933808e-01 -1.16780490e-01
 -7.96204656e-02  6.20231688e-01  5.61652891e-02 -1.11111887e-02
  1.79787904e-01  8.98607150e-02  4.38831061e-01  7.00074613e-01
  1.20653346e-01  3.17143410e-01  2.67180324e-01  3.00351650e-01
 -1.90911070e-01  5.33366919e-01  1.99243560e-01  5.93145907e-01
 -3.40085834e-01 -4.59780544e-02  2.09262654e-01 -9.68891904e-02
 -1.80566862e-01 -1.81853443e-01  9.19886827e-01  1.92857891e-01
  4.93163645e-01  4.21973646e-01 -6.11091815e-02 -2.44281311e-02
 -3.79558682e-01  2.94650048e-01 -2.51497030e-01 -2.12196499e-01
  3.99188474e-02  1.43035054e-01  4.35351253e-01  2.89106611e-02
  3.75047117e-01  7.72809237e-0